In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download en_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 1.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install pytextrank
!pip install sumy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/dataset_cleaned.csv")

In [ ]:
df

,title,author,published_date,genre,summary,published_year
0,Animal Farm,George Orwell,1945-08-17,"['Roman à clef', 'Satire', ""Children's literat...","Old Major, the old boar on the Manor Farm, ca...",1945
1,A Clockwork Orange,Anthony Burgess,1962,"['Science Fiction', 'Novella', 'Speculative fi...","Alex, a teenager living in near-future Englan...",1962
2,The Plague,Albert Camus,1947,"['Existentialism', 'Fiction', 'Absurdist ficti...",The text of The Plague is divided into five p...,1947
3,An Enquiry Concerning Human Understanding,David Hume,Unknown Date,['Unknown Genre'],The argument of the Enquiry proceeds by a ser...,Unkn
4,A Fire Upon the Deep,Vernor Vinge,Unknown Date,"['Hard science fiction', 'Science Fiction', 'S...",The novel posits that space around the Milky ...,Unkn
...,...,...,...,...,...,...
16550,Under Wildwood,Colin Meloy,2012-09-25,['Unknown Genre'],"Prue McKeel, having rescued her brother from ...",2012
16551,Transfer of Power,Vince Flynn,2000-06-01,"['Thriller', 'Fiction']",The reader first meets Rapp while he is doing...,2000
16552,Decoded,Jay-Z,2010-11-16,['Autobiography'],The book follows very rough chronological ord...,2010
16553,America Again: Re-becoming The Greatness We Ne...,Stephen Colbert,2012-10-02,['Unknown Genre'],Colbert addresses topics including Wall Stree...,2012


In [ ]:
# clean summarys
import re
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()

    # Remove special characters using regular expressions
    text = re.sub(r'[^a-zA-Z0-9\s.,?!;:]', '', text)

    return text

In [ ]:
# Apply preprocessing to each summary in the DataFrame
df['cleaned_summary'] = df['summary'].apply(preprocess_text)
# calculate each summary length before summarization
df['original_summary_length'] = df['cleaned_summary'].apply(lambda x: len(x))
df

,title,author,published_date,genre,summary,published_year,cleaned_summary,original_summary_length
0,Animal Farm,George Orwell,1945-08-17,"['Roman à clef', 'Satire', ""Children's literat...","Old Major, the old boar on the Manor Farm, ca...",1945,"old major, the old boar on the manor farm, ca...",5756
1,A Clockwork Orange,Anthony Burgess,1962,"['Science Fiction', 'Novella', 'Speculative fi...","Alex, a teenager living in near-future Englan...",1962,"alex, a teenager living in nearfuture england...",6059
2,The Plague,Albert Camus,1947,"['Existentialism', 'Fiction', 'Absurdist ficti...",The text of The Plague is divided into five p...,1947,the text of the plague is divided into five p...,6624
3,An Enquiry Concerning Human Understanding,David Hume,Unknown Date,['Unknown Genre'],The argument of the Enquiry proceeds by a ser...,Unkn,the argument of the enquiry proceeds by a ser...,16950
4,A Fire Upon the Deep,Vernor Vinge,Unknown Date,"['Hard science fiction', 'Science Fiction', 'S...",The novel posits that space around the Milky ...,Unkn,the novel posits that space around the milky ...,4446
...,...,...,...,...,...,...,...,...
16550,Under Wildwood,Colin Meloy,2012-09-25,['Unknown Genre'],"Prue McKeel, having rescued her brother from ...",2012,"prue mckeel, having rescued her brother from ...",956
16551,Transfer of Power,Vince Flynn,2000-06-01,"['Thriller', 'Fiction']",The reader first meets Rapp while he is doing...,2000,the reader first meets rapp while he is doing...,1228
16552,Decoded,Jay-Z,2010-11-16,['Autobiography'],The book follows very rough chronological ord...,2010,the book follows very rough chronological ord...,1830
16553,America Again: Re-becoming The Greatness We Ne...,Stephen Colbert,2012-10-02,['Unknown Genre'],Colbert addresses topics including Wall Stree...,2012,colbert addresses topics including wall stree...,147


In [ ]:
# making a subset of the main dataframe for faster processing on the next steps
df_subset = df.sample(n=30)
df_subset.reset_index(drop=True, inplace=True)
df_subset = pd.concat([df.iloc[0].to_frame().T, df_subset], ignore_index=True) # preserve first row

In [ ]:
df_subset

,title,author,published_date,genre,summary,published_year,cleaned_summary,original_summary_length
0,Animal Farm,George Orwell,1945-08-17,"['Roman à clef', 'Satire', ""Children's literat...","Old Major, the old boar on the Manor Farm, ca...",1945,"old major, the old boar on the manor farm, ca...",5756
1,The Wee Free Men,Terry Pratchett,2003,"['Speculative fiction', ""Children's literature...",An experienced witch named Miss Tick and her ...,2003,an experienced witch named miss tick and her ...,685
2,The Amber Room,Steve Berry,2003,"['Thriller', 'Mystery', 'Crime Fiction', 'Fict...",The story is about judge Rachel Cutler and he...,2003,the story is about judge rachel cutler and he...,528
3,Bull Run,Paul Fleischman,Unknown Date,"['Historical novel', ""Children's literature"", ...",The novel is a collection of monologues by si...,Unkn,the novel is a collection of monologues by si...,485
4,Under the Yoke,Unknown Author,Unknown Date,['Unknown Genre'],The tranquility in a Bulgarian village under ...,Unkn,the tranquility in a bulgarian village under ...,783
5,Evolution,Stephen Baxter,2002-11-30,"['Science Fiction', 'Speculative fiction', 'Fa...",The book follows the evolution of mankind as ...,2002,the book follows the evolution of mankind as ...,1741
6,Moving Pictures,Terry Pratchett,1990,"['Science Fiction', 'Speculative fiction', 'Fa...",The alchemists of the Discworld have invented...,1990,the alchemists of the discworld have invented...,997
7,Rite of Passage,Alexei Panshin,1968,"['Science Fiction', 'Speculative fiction', 'Fi...",Rite of Passage is told as a flashback by Mia...,1968,rite of passage is told as a flashback by mia...,5359
8,City of Thieves,Ian Livingstone,Unknown Date,"['Gamebook', ""Children's literature"", 'Fantasy...",The player takes the role of an adventurer on...,Unkn,the player takes the role of an adventurer on...,509
9,Behemoth,Scott Westerfeld,2010-10-05,"['Alternate history', 'Steampunk']",Behemoth continues an alternate history story...,2010,behemoth continues an alternate history story...,403


# **Extractive Summarization Method:**



---


Spacy Frequent based approch:

---





In [ ]:
import spacy
from collections import Counter

# Load spaCy model
nlp = spacy.load("en_core_web_trf")


def preprocess_text_spacy(text):
    # Tokenize text into words
    doc = nlp(text)
    # Get word frequencies
    word_freq = Counter(token.text for token in doc if not token.is_stop and not token.is_punct)
    return word_freq


def generate_summary(text):
    # Apply preprocessing using spaCy
    word_freq = preprocess_text_spacy(text)

    # Calculate total number of words
    total_words = sum(word_freq.values())

    # Calculate the desired number of words for the summary
    summary_length = int(total_words * 0.6)

    # Sort words by frequency
    sorted_word_freq = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)

    # Select top words to construct the summary
    summary_words = [word for word, freq in sorted_word_freq][:summary_length]


    summary_text = ' '.join(summary_words)

    return summary_text


# Apply preprocessing using spaCy to each summary in the DataFrame
df_subset['shortened_summary_spacy_freq'] = df_subset['cleaned_summary'].apply(generate_summary)


In [ ]:
df_subset

,title,author,published_date,genre,summary,published_year,cleaned_summary,original_summary_length,shortened_summary_spacy_freq
0,Animal Farm,George Orwell,1945-08-17,"['Roman à clef', 'Satire', ""Children's literat...","Old Major, the old boar on the Manor Farm, ca...",1945,"old major, the old boar on the manor farm, ca...",5756,animals napoleon farm animal pigs snowball c...
1,Arrow's Fall,Mercedes Lackey,1988-01,"['Speculative fiction', 'Fantasy']",Having completed her year and a half internin...,1988,having completed her year and a half internin...,3691,talia dirk elspeth kris ancar time hardorn you...
2,Harlequin,Bernard Cornwell,2000-10-16,['Historical novel'],;Prologue: The narrative begins in the villag...,2000,;prologue: the narrative begins in the villag...,6590,thomas sir jekyll guillaume jeanette french en...
3,Vulcan's Glory,D. C. Fontana,1989-02,['Science Fiction'],"The novel focuses on a young Spock, a conflic...",1989,"the novel focuses on a young spock, a conflic...",623,spock novel focuses enterprise soon mission ...
4,Gilda Joyce: Psychic Investigator,Jennifer Allison,2005-07,['Speculative fiction'],"Gilda Joyce is a spunky, 13-year-old that wan...",2005,"gilda joyce is a spunky, 13yearold that wants...",2436,gilda splinter summer san francisco splinters ...
5,Gossamer,Lois Lowry,Unknown Date,"['Fantasy', 'Science Fiction', 'Speculative fi...","The book's omniscient point of view, Littlest...",Unkn,"the books omniscient point of view, littlest ...",4834,littlest thin john elderly woman fragments dre...
6,Riders in the Chariot,Patrick White,1961,['Fiction'],The novel begins with the wild and mad Miss H...,1961,the novel begins with the wild and mad miss h...,481,factory novel begins wild mad miss hare awai...
7,Miracle Cure,Harlan Coben,Unknown Date,"['Thriller', 'Mystery']",The plot concerns a clinic that treats people...,Unkn,the plot concerns a clinic that treats people...,366,clinic plot concerns treats people aids scie...
8,Vox,Paul Stewart,2003-09-04,"['Speculative fiction', 'Fantasy', ""Children's...",The novel begins by saying that in the time s...,2003,the novel begins by saying that in the time s...,6232,rook librarians night storm vox sewers guard...
9,Tourist Season,Carl Hiaasen,1986,"['Crime Fiction', 'Mystery', 'Suspense']","Las Noches de Diciembre (Spanish, ""The Nights...",1986,"las noches de diciembre spanish, the nights o...",8713,keyes wiley las kara noches garcia lynn bernal...


In [ ]:
# add summary lengths after above approch
df_subset['shortened_summary_spacy_freq_length'] = df_subset['shortened_summary_spacy_freq'].apply(lambda x: len(x))

In [ ]:
df_subset.iloc[0]['shortened_summary_spacy_freq']

'animals napoleon farm animal pigs   snowball commandments shall squealer humans equal windmill seven old mr alcohol legs animalism attempts dogs changes harder better boxer van faces sleep bed drink major manor beasts england young philosophy jones writing food leadership announces pig neighbouring farmers control history glorifying replaced working care wear clothes having order original goes kill excess sheets boar calls meeting compares parasites teaches revolutionary song dies assume command turn dream revolt drive drunken irresponsible renaming adopt important teach reading plentiful runs smoothly elevate positions set aside special items ostensibly personal health takes pups trains privately struggle plans build chase away declares leader enacts governance structure replacing meetings committee run named mouthpiece claims credit idea work promise easier lives violent storm find annihilated convince destroyed scorn suggests walls thin scapegoat begins purging killing accuses cons



---


Spacy Text rank approch:

---





In [ ]:
import pytextrank

# Load spaCy model with TextRank extension
nlp = spacy.load("en_core_web_trf")
nlp.add_pipe("textrank")

def generate_summary(text):

    doc = nlp(text)

    # Initialize variables to track summary length and desired length
    summary = ''
    summary_length = 0
    desired_length = len(text) * 0.6  # Desired summary length is 60% of original text length

    # Generate the summary by iterating through the sentences
    for sent in doc._.textrank.summary(limit_phrases=20, limit_sentences=20):
        # Add the sentence to the summary
        summary += sent.text
        summary_length += len(sent.text)

        # Check if the summary length exceeds 60% of the original text length
        if summary_length >= desired_length:
            break  # Stop adding sentences to the summary

    return summary

# Generate the summary
df_subset['shortened_summary_spacy_textrank'] = df_subset['cleaned_summary'].apply(generate_summary)


/usr/local/lib/python3.10/dist-packages


In [ ]:
# add summary lengths after above approch
df_subset['shortened_summary_spacy_textrank_length'] = df_subset['shortened_summary_spacy_textrank'].apply(lambda x: len(x))

In [ ]:
df_subset.iloc[0]['shortened_summary_spacy_textrank']

'this is an ironic twist to the original purpose of the seven commandments, which were supposed to keep order within animal farm by uniting the animals together against the humans, and prevent animals from following the humans evil habits.once snowball becomes a scapegoat, napoleon begins purging the farm with his dogs, killing animals he accuses of consorting with his old rival.the animals revolt and drive the drunken and irresponsible mr jones from the farm, renaming it animal farm.during a poker match, an argument breaks out between napoleon and mr pilkington, and the animals realise that the faces of the pigs look like the faces of humans, and no one can tell the difference between them.napoleon and squealer convince the animals that snowball destroyed it, although the scorn of the neighbouring farmers suggests that its walls were too thin.the pigs snowball, napoleon, and squealer adapt old majors ideas into an actual philosophy, which they formally name animalism.soon after, napol



---


Sumy LSA approch:

---





In [ ]:
import nltk

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer


# Function to apply LSA summarization to each text document
def apply_lsa_summarization(text):
    # Initialize parser and tokenizer
    parser = PlaintextParser.from_string(text, Tokenizer("english"))

    # Initialize LSA summarizer
    summarizer = LsaSummarizer()

    # Calculate desired summary length (60% of original text length)
    desired_length = int(len(text) * 0.6)

    # Summarize the text using LSA with a large number of sentences
    summary_sentences = summarizer(parser.document, sentences_count=20)  # Use a large number for sentences_count

    # Track summary length
    summary_length = 0

    # Select sentences for summary until desired length is reached
    selected_sentences = []
    for sentence in summary_sentences:
        if summary_length + len(str(sentence)) <= desired_length:
            selected_sentences.append(sentence)
            summary_length += len(str(sentence))
        else:
            break

    # Convert selected summary sentences to string
    summary_text = ' '.join(str(sentence) for sentence in selected_sentences)

    return summary_text

# Apply LSA summarization to each text document in the DataFrame
df_subset['shortened_summary_sumy_lsa'] = df_subset['cleaned_summary'].apply(apply_lsa_summarization)


In [ ]:
df_subset

,title,author,published_date,genre,summary,published_year,cleaned_summary,original_summary_length,shortened_summary_spacy_freq,shortened_summary_spacy_freq_length,shortened_summary_spacy_textrank,shortened_summary_spacy_textrank_length,shortened_summary_sumy_lsa
0,Animal Farm,George Orwell,1945-08-17,"['Roman à clef', 'Satire', ""Children's literat...","Old Major, the old boar on the Manor Farm, ca...",1945,"old major, the old boar on the manor farm, ca...",5756,animals napoleon farm animal pigs snowball c...,2351,this is an ironic twist to the original purpos...,2696,"when major dies, two young pigs, snowball and ..."
1,Arrow's Fall,Mercedes Lackey,1988-01,"['Speculative fiction', 'Fantasy']",Having completed her year and a half internin...,1988,having completed her year and a half internin...,3691,talia dirk elspeth kris ancar time hardorn you...,1416,talia and kris who has resolved his quarrel wi...,2322,having completed her year and a half interning...
2,Harlequin,Bernard Cornwell,2000-10-16,['Historical novel'],;Prologue: The narrative begins in the villag...,2000,;prologue: the narrative begins in the villag...,6590,thomas sir jekyll guillaume jeanette french en...,2316,jekyll encounters thomas on the battlefield an...,1977,"thomas has been training secretly with a bow, ..."
3,Vulcan's Glory,D. C. Fontana,1989-02,['Science Fiction'],"The novel focuses on a young Spock, a conflic...",1989,"the novel focuses on a young spock, a conflic...",623,spock novel focuses enterprise soon mission ...,258,"the novel focuses on a young spock, a conflict...",396,"the novel focuses on a young spock, a conflict..."
4,Gilda Joyce: Psychic Investigator,Jennifer Allison,2005-07,['Speculative fiction'],"Gilda Joyce is a spunky, 13-year-old that wan...",2005,"gilda joyce is a spunky, 13yearold that wants...",2436,gilda splinter summer san francisco splinters ...,904,thrilled at this opportunity for investigation...,1569,"gilda joyce is a spunky, 13yearold that wants ..."
5,Gossamer,Lois Lowry,Unknown Date,"['Fantasy', 'Science Fiction', 'Speculative fi...","The book's omniscient point of view, Littlest...",Unkn,"the books omniscient point of view, littlest ...",4834,littlest thin john elderly woman fragments dre...,1866,"on the way, thin elderly explains to be gentle...",2246,"at their home, the heap, fastidious complains ..."
6,Riders in the Chariot,Patrick White,1961,['Fiction'],The novel begins with the wild and mad Miss H...,1961,the novel begins with the wild and mad miss h...,481,factory novel begins wild mad miss hare awai...,163,the novel begins with the wild and mad miss ha...,360,the novel begins with the wild and mad miss ha...
7,Miracle Cure,Harlan Coben,Unknown Date,"['Thriller', 'Mystery']",The plot concerns a clinic that treats people...,Unkn,the plot concerns a clinic that treats people...,366,clinic plot concerns treats people aids scie...,125,pl:klinika mierciinitially it looks like suici...,238,the plot concerns a clinic that treats people ...
8,Vox,Paul Stewart,2003-09-04,"['Speculative fiction', 'Fantasy', ""Children's...",The novel begins by saying that in the time s...,2003,the novel begins by saying that in the time s...,6232,rook librarians night storm vox sewers guard...,2426,rook realizes that vox intended to doublecross...,3803,the guardians of night are awaiting a storm wh...
9,Tourist Season,Carl Hiaasen,1986,"['Crime Fiction', 'Mystery', 'Suspense']","Las Noches de Diciembre (Spanish, ""The Nights...",1986,"las noches de diciembre spanish, the nights o...",8713,keyes wiley las kara noches garcia lynn bernal...,3775,"at first, wiley refuses to tell, prepared to l...",3267,"las noches de diciembre spanish, the nights of..."


In [ ]:
# add summary lengths after above approch
df_subset['shortened_summary_sumy_lsa_length'] = df_subset['shortened_summary_sumy_lsa'].apply(lambda x: len(x))

In [ ]:
df_subset.iloc[0]['shortened_summary_sumy_lsa']

'when major dies, two young pigs, snowball and napoleon, assume command and turn his dream into a philosophy. snowball attempts to teach the animals reading and writing; food is plentiful, and the farm runs smoothly. the pigs elevate themselves to positions of leadership and set aside special food items, ostensibly for their personal health. when snowball announces his plans to build a windmill, napoleon has his dogs chase snowball away and declares himself leader. once snowball becomes a scapegoat, napoleon begins purging the farm with his dogs, killing animals he accuses of consorting with his old rival. squealer justifies every statement napoleon makes, even the pigs alteration of the seven commandments of animalism to benefit themselves. beasts of england is replaced by an anthem glorifying napoleon, who appears to be adopting the lifestyle of a man. squealer abuses the animals poor memories and invents numbers to show their improvement. despite his injuries, boxer continues workin

In [ ]:
# save the result of the Extractive methods
df_subset.to_csv('/content/drive/MyDrive/dataset_summarized.csv')

# Abstractive Method:

In [ ]:
!pip install git+https://github.com/PyTorchLightning/pytorch-lightning
!pip install git+https://github.com/huggingface/transformers
!pip install sentencepiece
!pip install git+https://github.com/stas00/transformers
!pip install pegasus

  Cloning https://github.com/PyTorchLightning/pytorch-lightning to /tmp/pip-req-build-7xg89rim
  Running command git clone --filter=blob:none --quiet https://github.com/PyTorchLightning/pytorch-lightning /tmp/pip-req-build-7xg89rim
  Resolved https://github.com/PyTorchLightning/pytorch-lightning to commit 5e0e02b79e07b1c7f76d75d2fcb669dbc13e958e
  Running command git submodule update --init --recursive -q
  Encountered 22 file(s) that should have been pointers, but weren't:
        .notebooks/course_UvA-DL/01-introduction-to-pytorch.ipynb
        .notebooks/course_UvA-DL/02-activation-functions.ipynb
        .notebooks/course_UvA-DL/03-initialization-and-optimization.ipynb
        .notebooks/course_UvA-DL/04-inception-resnet-densenet.ipynb
        .notebooks/course_UvA-DL/05-transformers-and-MH-attention.ipynb
        .notebooks/course_UvA-DL/06-graph-neural-networks.ipynb
        .notebooks/course_UvA-DL/07-deep-energy-based-generative-models.ipynb
        .notebooks/course_UvA-DL/08-

  Cloning https://github.com/stas00/transformers to /tmp/pip-req-build-zrmhza8_
  Running command git clone --filter=blob:none --quiet https://github.com/stas00/transformers /tmp/pip-req-build-zrmhza8_
  Resolved https://github.com/stas00/transformers to commit 59d5edef34ae0fa56065a2e863736d4f133c558b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 29.7 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.27.0.dev0-py3-none-any.whl size=6434362 sha256=a01b2f7e2dc47208c4a22aed896d81322f7b3a2f0e43cc37db49c055593e94cc
  Stored in directory: /tmp/pip-ephem-wheel-cache-mya23o7i/wheels/a4/d2/27/008088738c12a0365dfc849aa1725015edb10e86f5eb5cbb12
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled to

  Preparing metadata (setup.py) ... done
  Created wheel for pegasus: filename=pegasus-0.1.3-py3-none-any.whl size=6466 sha256=8ced2a90a3884ff49ee191f2fdd1ec4b1c940cb8071b22296525dca32fcdee05
  Stored in directory: /root/.cache/pip/wheels/dc/cb/73/ee2d4013c3271663234bb15f430ba7597d48c0480528998901
Successfully built pegasus


In [ ]:
from transformers import pipeline
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

# Pick model
model_name = "google/pegasus-xsum"
# Load pretrained tokenizer
pegasus_tokenizer = PegasusTokenizer.from_pretrained(model_name)

example_text = df_subset.iloc[2]['summary']

print('Original Document Size:',len(example_text))
# Define PEGASUS model
pegasus_model = PegasusForConditionalGeneration.from_pretrained(model_name)
# Create tokens
tokens = pegasus_tokenizer(example_text, truncation=True, padding="longest", return_tensors="pt")

# Generate the summary
encoded_summary = pegasus_model.generate(**tokens)

# Decode the summarized text
decoded_summary = pegasus_tokenizer.decode(encoded_summary[0], skip_special_tokens=True)

# Print the summary
print('Decoded Summary :',decoded_summary)

summarizer = pipeline(
	"summarization",
	model=model_name,
	tokenizer=pegasus_tokenizer,
	framework="pt"
)

summary = summarizer(example_text, min_length=30, max_length=150)
summary[0]["summary_text"]


Original Document Size: 395


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: Using `max_length`'s default (64) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Decoded Summary : The Dark God Naar is back!


Your max_length is set to 150, but you input_length is only 87. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=43)


"The Dark God Naar is back, and this time he's trying to take down the Kai Grand Master himself, Lone Wolf, and his allies."

# Measure Summarization Quality:

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

# Compute BLEU score
bleu_score_spacy = sentence_bleu([df_subset.iloc[0]['cleaned_summary'].split()], df_subset.iloc[0]['shortened_summary_spacy_freq'].split())
bleu_score_textrank = sentence_bleu([df_subset.iloc[0]['cleaned_summary'].split()], df_subset.iloc[0]['shortened_summary_spacy_textrank'].split())
bleu_score_lsa = sentence_bleu([df_subset.iloc[0]['cleaned_summary'].split()], df_subset.iloc[0]['shortened_summary_sumy_lsa'].split())
# Print BLEU score
print("BLEU score Spacy:", bleu_score_spacy)
print("BLEU score: Text Rank", bleu_score_textrank)
print("BLEU score LSA", bleu_score_lsa)

BLEU score Spacy: 2.3067912357133334e-79
BLEU score: Text Rank 0.25232056497971667
BLEU score LSA 0.20583514202055556


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
